In [7]:
import random

import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import pickle

import signature
import datasets
import utils

In [2]:
data = datasets.TDSCTumors()

In [5]:
def calculate_signature(m):
    slice_signatures = []
    d, _, _ = m.shape
    for i in range(d):
        s = m[i, :, :]
        slice_signatures += [signature.signature(s, res=1)]
    return slice_signatures

In [6]:
signature_data = []
try:
    with open("./data/tdsc/tumors/signature.pkl", "rb") as sig_file:
        signature_data = pickle.load(sig_file)
except IOError:
    for datum in tqdm(data):
            _, x,y = datum
            sig = calculate_signature(x)
            signature_data += [(sig, y)]
    with open("./data/tdsc/tumors/signature.pkl", "wb") as sig_file:
        pickle.dump(signature_data, sig_file)
    
        
for i in range(len(signature_data)):
    x, y = signature_data[i]
    num_zero_slices = 180 - len(x)
    zero_slices = [np.zeros(360, dtype=np.float32) for i in range(num_zero_slices)]
    x = x + zero_slices
    x=np.array(x, dtype=np.float32)
    y=np.array(y, dtype=np.float32)
    x = torch.tensor(x)
    y = torch.tensor(y).unsqueeze(0)
    signature_data[i] = (x,y)

In [8]:
malignant_signatures = range(0, 58)
benign_signatures = range(58, 100)
all_tumors = range(100)

test_set = random.sample(malignant_signatures, 10) + random.sample(benign_signatures, 10)
train_set = list(set(all_tumors) - set(test_set))



[1, 16, 41, 8, 17, 13, 57, 27, 56, 22, 72, 94, 59, 63, 70, 82, 84, 99, 80, 89]
[0, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 14, 15, 18, 19, 20, 21, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 58, 60, 61, 62, 64, 65, 66, 67, 68, 69, 71, 73, 74, 75, 76, 77, 78, 79, 81, 83, 85, 86, 87, 88, 90, 91, 92, 93, 95, 96, 97, 98]


In [11]:
train_data = []
test_data = []

for idx in test_set:
    test_data.append(signature_data[idx])

for idx in train_set:
    train_data.append(signature_data[idx])

In [17]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=4, num_workers=8, shuffle=True, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=4, num_workers=8, shuffle=True, pin_memory=True)

In [18]:
class SequenceClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, device='cpu'):
        super(SequenceClassifier, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size * 180, num_classes)
        
    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)

        # Forward propagate LSTM
        out, _ = self.lstm(
            x, (h0, c0)
        )  # out: tensor of shape (batch_size, seq_length, hidden_size)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

In [21]:
num_epochs = 100
learning_rate = 10e-4
device = 'cuda'
criterion = torch.nn.CrossEntropyLoss()
model = SequenceClassifier(360, 180, 1, 2, device=device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [23]:
for epoch in range(num_epochs):
    print(f"Epoch #{epoch} training...")
    data_loop = tqdm(train_loader)
    for data in data_loop:
        
        x, y = data
        x = x.to(device).squeeze(1)
        y = y.to(device).squeeze(1).long()  
                
        predictions = model(x)
        loss = criterion(predictions, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        data_loop.set_postfix(loss=loss.item())
            
    total = 0
    num_corrects = 0
    
    for data in tqdm(train_loader):
        
        x, y = data
        x = x.to(device).squeeze(1)
        y = y.to(device).squeeze(1).long()  
        model.eval()
        
        with torch.no_grad():
        
            predictions = model(x)
            _, predictions = predictions.max(1)
            num_corrects += (predictions == y).float().sum()
            total += predictions.size(0)
        
        model.train()
        
    print(f"Epoch #{epoch} train acc is: {(num_corrects/total)*100}")
    
    for data in tqdm(test_loader):
        
        x, y = data
        x = x.to(device).squeeze(1)
        y = y.to(device).squeeze(1).long()  
        model.eval()
        
        with torch.no_grad():
        
            predictions = model(x)
            _, predictions = predictions.max(1)
            num_corrects += (predictions == y).float().sum()
            total += predictions.size(0)
        
        model.train()
            
    print(f"Epoch #{epoch} test acc is: {(num_corrects/total)*100}")

Epoch #0 training...


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch #0 train acc is: 86.25


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch #99 train acc is: 81.25


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch #99 test acc is: 74.0
